In [10]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}



<IPython.core.display.Javascript object>

 # Project: Adopt a Drain Data Cleaning
 * Author: James Wilfong, wilfongjt@gmail.com
 * This script doesn't interface with GitHub or Data.world. That happens in the maintainer/maintainer.Process.ipynb script.
 * The input data is contained in the raw-data folder of this repo. 
 * The cleaning process is initiated by running this Jupyter Notebook.
 * The cleaned data is put in the clean-data folder of this repo.
 * Clean data is pushed to repo by the developer.
 * The developer creates a pull request for the maintainer.
 * The Maintainer will complete the task of loading data into production

# IMPORTANT
## Configuring the Data Transfer
Configure before running "RUN All" in the Cell menu

In [11]:
import helper
'''
    Input: CSV file in raw_data/ folder
    Process: clean (conform, condence)
    Output: is directed to the clean-data/ folder
    
    Name the output file using OUTPUT_FILE
    OUTPUT_FILELOCAL_CLEAN_NAME is used to name the data.world table
    Table names should start with letter, may contain letters, numbers, underscores
    
'''
INPUT_FILE_NAME='grb_drains.csv' # input file is found in raw-data/ folder
OUTPUT_FILE_NAME='grb_drains.csv' # output file is found in clean-data/ folder. Name is used as data.world table name
gh_file_type = 'csv'
title = 'GRB Storm Drains' # title of data.world dataset. title is also used to name d.w data service
desc = 'Storm Drains of the Grand River Basin, Michigan' # describes contents of dataset, appears in d.w  
table_name=OUTPUT_FILE_NAME.replace('.csv','')
# write the maintainer_config to a file for the maintainer
helper.exportMaintainerConfig(OUTPUT_FILE_NAME, gh_file_type, title, desc, table_name)    


kill log:  ./maintainer/maintainer-config.json


In [12]:
'''
    Assemble Names of:
        Application,
        Raw data file,
        Clean data file
'''
LOCAL_RAW_FILE = helper.get_raw_data_folder() + '/{}'.format(INPUT_FILE_NAME )
LOCAL_CLEAN_FILE = helper.get_clean_data_folder() + '/{}'.format(OUTPUT_FILE_NAME)
local_config = { 
                 "app_name": helper.get_app_name(),
                 "local_raw": LOCAL_RAW_FILE,
                 "local_clean": LOCAL_CLEAN_FILE,
               }

'''
    ------------- configure outliers
'''
_outliers = {
  'outliers': [
    {'column':'dr_facility_id',
     'range':(1, 50000000),
     'reason':'ignore {} outliers (1 <= dr_facility_id or => 50000000).',
     'count': 0
    }, 
    {'column':'dr_lon',
     'range':(-90.0, -80.0),
     'reason':'Remove {} observations too far west or east.',
     'count': 0
    },  
    {'column':'dr_lat',
     'range':(40.0, 50.0),
     'reason':'Remove {} observations too far north or south.',
     'count': 0
    }
  ]
}  
ENV_ERROR=False

In [13]:
# %matplotlib notebook
from IPython.display import display, HTML
from IPython.display import Markdown

from lib.p3_ProcessLogger import ProcessLogger
cell_log = ProcessLogger() 

In [14]:
cell_log.clear()
import interface
cell_log.collect('* Import third party packages')

import numpy as np 
import pandas as pd

import pprint

import csv # read and write csv files

from pprint import pprint

import os
# import subprocess

# convenience functions -- cleaning
cell_log.collect('* Import custom packages')
from lib.p3_CellCounts import CellCounts
import lib.p3_clean as clean
from lib.p3_configuration import get_configuration
import lib.p3_explore as explore
import lib.p3_gather as gather # gathering functions
# import lib.p3_helper_functions as helper
import lib.p3_map as maps

if ENV_ERROR:
    cell_log.collect("# Script Failure!!")
    cell_log.collect("# !!! Missing Environment Variables !!!")
    cell_log.collect("### see [Environment Variable Setup](#env-setup)")

Markdown('''{}'''.format(cell_log.getMarkdown()))

* Import third party packages
* Import custom packages

## Wrangling Script

In [15]:
cell_log.clear()
if ENV_ERROR:
    cell_log.collect("# Script Failure!!")
    cell_log.collect("# !!! Missing Environment Variables !!!")
    cell_log.collect("### see [Environment Variable Setup](#env-setup)")
else:
    cell_log.collect("# CSV Process: What Happened?")
    '''
    --------------------------------- input
    '''
    print(local_config["local_raw"])
    cell_log.collect("* input:  {}".format( local_config["local_raw"]))
    '''
    --------------------------------- load data
    '''
    df_source = helper.open_raw_data(local_config) # open raw-data
    
    cell_log.collect("* input: {} observations".format(len(df_source)))
    cell_log.collect("* input: columns {}".format(df_source.columns.values))

    '''
    --------------------------------- clean column names
    '''
    cell_log.collect('* format: Apply a style of lowercase and underscores to column names.')##############################
    df_source = clean.clean_column_names(df_source) # column names

    '''
    --------------------------------- map expected colums to raw-data columns
    '''
    df_source = df_source.rename(columns={ # rename columns in df
        "subtype": "dr_subtype",
        "drain__owner": "dr_owner",
        "local__id": "dr_local_id",
        "facilityid": "dr_facility_id",
        "drain__jurisdiction": "dr_jurisdiction",
        "subwatershed": "dr_subwatershed",
        "point__x":"dr_lon", 
        "point__y":"dr_lat"})

    '''
    --------------------------------- change empty values
    '''

    ## ------------------------------ DROP empty Facility id
    # mark all empties with same value
    df_source['dr_facility_id'] = df_source['dr_facility_id'].apply(lambda x:  np.nan if x != x or x == '' or x == ' ' or x == None else x)
    scnt = len(df_source)
    df_source = df_source.dropna(subset=['dr_facility_id', 'soure__id','dr_lon', 'dr_lat'])
    ecnt = len(df_source)
    cell_log.collect("* clean: dropped {} observations with empty dr_facility_id, soure___id, dr_lon, or dr_lat".format(scnt - ecnt))

    '''
    --------------------------------- change column types
    '''
    cell_log.collect('* format: convert dr_facility_id column to int64')
    df_source['dr_facility_id'] = df_source['dr_facility_id'].astype('int64')

    '''
    --------------------------------- remove numbers from df_source_id
    '''

    df_source['soure__id'] = df_source['soure__id'].apply(lambda x: x.split('_')[0] + '_' if isinstance(x, str) else 'XXX_') 

    df_source['dr_asset_no'] = df_source['dr_facility_id']
    df_source['dr_type'] = df_source['dr_facility_id'].apply(lambda x: 'Storm Water Inlet Drain')
    '''
    --------------------------------- create a sync id
    '''
    df_source['dr_sync_id'] = df_source['soure__id'] + df_source['dr_facility_id'].astype(str)

    '''
    --------------------------------- drop soure__id
    '''
    df_source = df_source.drop(['soure__id'], axis=1)

    '''
    --------------------------------- outliers
    '''
    df_source = clean.remove_obvious_outliers(_outliers, df_source)
    for r in _outliers['outliers']:
        cell_log.collect('* outlier: {}'.format(r['reason']))

    '''
    --------------------------------- Drop DUPLICATES
    '''
    scnt = len(df_source)
    df_source = df_source.drop_duplicates('dr_facility_id',keep=False)
    ecnt = len(df_source)
    cell_log.collect('* duplicates: dropped {} duplicate facility ids'.format(scnt - ecnt))

    '''
    --------------------------------- save csv 
    '''
    # assume new file and remove old one
    if os.path.isfile(local_config["local_clean"]):
        os.remove(local_config['local_clean'])
        cell_log.collect('* deleted {} '.format(local_config['local_clean']))
    
    cell_log.collect("* inter-output: columns {}".format(df_source.columns.values))
    cell_log.collect('* inter-output: {} obs to {}'.format(len(df_source) , local_config["local_clean"]))

    df_source.to_csv(local_config["local_clean"], index=False)

Markdown('''{}'''.format(cell_log.getMarkdown()))

/Users/jameswilfong/Documents/Github/CitizenLabs/data.world/raw-data/adopt-a-drain/catch basins.csv
* clean_column_names: 0.004213094711303711 sec
* remove_obvious_outliers: 0.009351015090942383 sec


# CSV Process: What Happened?
* input:  /Users/jameswilfong/Documents/Github/CitizenLabs/data.world/raw-data/adopt-a-drain/catch basins.csv
* input: 40204 observations
* input: columns ['SUBTYPE' 'DRAIN_JURISDICTION' 'DRAIN_OWNER' 'Soure_ID' 'LOCAL_ID'
 'FACILITYID' 'Subwatershed' 'POINT_X' 'POINT_Y']
* format: Apply a style of lowercase and underscores to column names.
* clean: dropped 15 observations with empty dr_facility_id, soure___id, dr_lon, or dr_lat
* format: convert dr_facility_id column to int64
* outlier: ignore 0 outliers (1 <= dr_facility_id or => 50000000).
* outlier: Remove 0 observations too far west or east.
* outlier: Remove 0 observations too far north or south.
* duplicates: dropped 225 duplicate facility ids
* inter-output: columns ['dr_subtype' 'dr_jurisdiction' 'dr_owner' 'dr_local_id' 'dr_facility_id'
 'dr_subwatershed' 'dr_lon' 'dr_lat' 'dr_asset_no' 'dr_type' 'dr_sync_id']
* inter-output: 39964 obs to /Users/jameswilfong/Documents/Github/CitizenLabs/data.world/clean-data/adopt-a-drain/grb_drains.csv

# Appendix - Data.World Names

## Keeping the names straight

| CSV Name      | Table Name    | Title          | Dataset ID      | Restful |
| :------------ |:------------- | :------------- | :-------------  | :------------- |
| xxxx_xx       | xxxx_xx       | Xxxx Xx        | xxxx-xx         |    ?     | 
| xxxx_xx       | xxxx_xx       | Xxxx_Xx        | xxxxxx          |    ?            |
| xxxx_xx       | xxxx_xx       | Xxxx-Xx        | xxxx-xx         |    ?         |
| xxxx-xx       | xxxx_xx       | Xxxx Xx        | xxxx-xx         |    ?         |
| xxxx-xx       | xxxx_xx       | Xxxx_Xx        | xxxxxx          |    ?         |
| xxxx-xx       | xxxx_xx       | Xxxx-Xx        | xxxx-xx         |    ?         |

* CSV Name is root of Table name
* Title is root of Dataset ID
* a space in Title will be automatically converted to hyphen in dataset id
* an underscore in Title will be removed in Dataset ID
* a hyphen in CSV Name will be replaced with underscore in Table Name
